# Learning with Gradient Descent Optimization

In this notebook you will implement a simple machine learning algorithm yourself and then see how the same model can be implemented in Tensorflow.

In [ ]:
%matplotlib inline

from sklearn.datasets import make_blobs
from sklearn.cross_validation import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

### Dataset

We use scikit-learn to create a simple artificial dataset that consists of two clusters. Our goal in this toy problem is to learn to distinguish "red" points from "blue" points. (By the way, scikit-learn has many toy datasets built-in and they are great for building intuition about how different methods work.)

In [ ]:
# generate a small "blobs" dataset
X, y = make_blobs(n_samples=1000, n_features=2, cluster_std=1.0, centers=[(-2, 0), (2, 0)], random_state=42)

print 'Vectors: \n', X[:10]
print 'Labels: \n', y[:10]

Our dataset consists of 1000 points each with 2 dimensions. Each point has an associated label (0 vs. 1) indicating which blob that point belongs to. Since we have labels for what we are trying to predict from the data (which blob the point belongs to), this is called a **supervised** problem.

In [ ]:
# plot the data points
x_min = y_min = -5
x_max = y_max = 5
plt.figure(figsize=(10, 10))
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, linewidths=0)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.axes().set_aspect('equal')

Inpecting the dataset, we can see that this is an "easy" dataset because the points on are only 2 dimensions and the blobs are, roughly speaking, **linearly separable** - this means that we can do well on the problem by finding a line that divides the two blobs and making a decision by simply finding what side of the line each point is on.

Next we will split our data into training and test sets.

In [ ]:
# split into training and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=45)

## Train a linear classifier with Python

We use a logistic regression classifier, which we optimize using stochastic gradient descent. We provide below helper functions to compute the estimated probability, loss, gradients, and classifier accuracy. 

### Exercise 0 - define model functions

We need to make some basic decisions about the structure of our model. We will first do this in python before moving our model to Tensorflow. Below are four functions we need to define along with tests to let you know if you have it correct. Using the mathematical definitions of those functions below, fill in the correct python code for each function.

**1) Probability**

First we need to define how our model computes $p(y=1|x)$, the probability of a point belonging to blob 1 (the probability of belong to blob 0 is just $1-p(y=1|x)$. In our model, this is defined as $p(y=1|x) = \sigma(xW + b)$ where $\sigma$ is the **logistic sigmoid** activation function, $\sigma(t) = \frac{1}{1 + e^{-t}}$.

Fill in `def prob(X, w, b)` below to implement this function given a matrix of examples `X`, a matrix of weights `w`, and a vector of biases `b`.

Hint: you will need to use the `np.dot` function for weight matrix multiplication and `np.exp` for the exponential.

**2) Prediction**

Once we have a probability, we need a function to turn it into a decision. In this case, we should return the integer label we predict for each example, which is simply $1$ if $p(y=1|x) > 0.5$ and $0$ otherwise.

Complete `def predict(X, w, b)` to return a decision for each example.

**3) Loss**

In order to learn, we must define a **loss** function that gives us a score telling us how well our model is doing. For this problem, we will use **sigmoid cross-entropy** loss which is defined as $L =-y \log(p) - (1-y)\log(1 - p)$ where $y$ is the label for an example (0 or 1) and $p$ is the probability our model assigns to the example.

You can learn more about cross-entropy loss [here](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression), though you won't need this to complete the exercise.

**4) Parameter update**

Now that we have the loss, we can compute the **gradients** of the loss with respect to our parameters which tells us how to change the parameters to improve performance. We have two different sets of parameters: the weights and biases. The gradient of the loss function with respect to each of these is, repectively, $\frac{\partial L}{\partial w} = -(y-p)x$ and $\frac{\partial L}{\partial b} = -(y-p)$.



In [ ]:
# utility functions

def prob(X, w, b):
    """
    Compute posterior probability p(y=1|x)
    """
    # TODO - implement probability function
    return ...
    
def predict(X, w, b):
    """
    Predict labels
    """
    p = prob(X, w, b)
    
    # TODO - implement prediction function
    return ...

def loss(X, y, w, b):
    """
    Compute loss function
    """
    # compute posterior probability
    p = prob(X, w, b)

    # TODO - compute cross-entropy loss
    l = ...
    
    return l.mean()

def grad_loss(x, y, w, b):
    """
    Compute gradient of the loss
    """
    p = prob(x, w, b)

    # TODO - compute gradients for weights and biases
    dLdw = ...
    dLdb = ...

    return dLdw, dLdb



# generate test data
np.random.seed(42)
rX = np.random.randn(1, 2)
ry = 0
rw = np.random.randn(2, 1)
rb = np.random.randn(1)

print 'Test passing for "prob": ', np.allclose(prob(rX, rw, rb), 0.46928423)
print 'Test passing for "predict": ', np.allclose(predict(rX, rw, rb), 0)
print 'Test passing for "loss": ', np.allclose(loss(rX, ry, rw, rb), 0.63352868)
gw, gb = grad_loss(rX, ry, rw, rb)
print 'Test passing for "grad_loss": ', np.allclose(gw, np.array([[ 0.23310012, -0.06488526]])) and \
                                        np.allclose(gb, 0.46928423)


### Exercise 1 - implement the parameter update in the training loop

Below is the training loop for the logistic regression classifier we defined above. Before running the code, you need to implement one last detail: the paramter update itself. To do **gradient descent**, we need to take the gradient for the parameters, multiple that gradient by a **learning rate** parameter (below called `eta`) to get the update, and modify the parameters with that update. The different between mimimizing the loss function and maximizing the loss function is the difference between subtracting the update and adding the update.

Fill in the update equations below and train the model. How well does your model do on the test dataset?

In [ ]:
# initialize classifier parameters
w = np.random.random((2,))
b = np.random.random((1,))

# learning rate
eta = 0.01

# number of iterations
num_iterations = 500

for i in range(num_iterations):
    
    # select training sample
    idx = i % X_train.shape[0]
    
    # compute gradients with respect to parameters for this example
    dw, db = grad_loss(X_train[idx], y_train[idx], w, b)
    
    # TODO - write update rule for w and b
    ...
    
    # print out performance every 20 iterations
    if i % (num_iterations / 20) == 0:
        y_train_hat = predict(X_train, w, b)
        y_test_hat = predict(X_test, w, b)
        print "iter: %04d: loss(train) = %.3f, acc(train) = %.3f, loss(test)= %.3f, acc(test) = %.3f" % \
            (i, loss(X_train, y_train, w, b), (y_train == y_train_hat).mean(), 
             loss(X_test, y_test, w, b), (y_test == y_test_hat).mean())


# visualize the decision boundary
h = 0.02
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
G = np.c_[xx.ravel(), yy.ravel()]
G_prob = prob(G, w, b)
G_prob = G_prob.reshape(xx.shape)

plt.figure(figsize=(10, 10))
plt.contourf(xx, yy, G_prob, cmap=plt.cm.RdBu_r, alpha=.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, linewidths=0)
plt.axes().set_aspect('equal')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max);


## Train logistic regression classifier in TensorFlow

Here we will train the same classifier as above, but we will be using TensorFlow and make use of automatic differentiation.

Tensorflow (and other deep learning frameworks like it) takes away a lot of the manual effort in defining losses, update formulas, etc. This frees us to focus more on the modeling and less on the mechanics.

### Exercise 2 - Tensorflow model parts

In the block below, we define all the pieces of our model. Find the following components:
1. The `tf.placeholder` defines the inputs to our model. In this case, we have two. What are they?
2. The `tf.Variable` defines the learnable parameters of our model. What are they? How are they initialized here?


### Exercise 3 - Define the model

Below you will fill in the same components as you did above, but you will use Tensorflow.

1. First define the model. This should be a single line that uses `tf.matmul` to multiply the input data `x_` with the weights `w_` then add the bias `b_`. Assign this to the `z_` variable.
2. Next define the prediction function. It should take `z_` as input and compute the probability using the sigmoid function. You will need to use `tf.exp`, but the rest of the operations can be written like in normal python.
3. Finally, you can use Tensorflow's built-in ["sigmoid cross entropy" loss function](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#sigmoid_cross_entropy_with_logits). This function takes `z_` as input (called the "logits") and the target output, `y_`, and returns the loss.

After doing the above, you are ready to train the model with Tensorflow! Notice that you didn't have to define the gradient of the loss. This is one of the things that Tensorflow can figure out for you.


In [ ]:
tf.reset_default_graph()

# placeholders define the inputs to our model
x_ = tf.placeholder(tf.float32, [None, 2])
y_ = tf.placeholder(tf.float32, [None, 1])

# variables define the parameters of out model
w_ = tf.Variable(tf.zeros([2, 1]))
b_ = tf.Variable(tf.zeros([1]))


# TODO - define the model
z_ = ...

# TODO - define the prediction function
p_ = ...

# TODO - define the loss function
loss_ = ...


loss_ = tf.reduce_mean(loss_)

# compute accuracy
y_hat_ = tf.cast(p_ > .5, "float")
correct_prediction_ = tf.equal(y_hat_, y_)
accuracy_ = tf.reduce_mean(tf.cast(correct_prediction_, "float"))

Train the model you defined above by running the block below.

In [ ]:
eta = 0.5
num_iterations = 100

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss_)
# train_step = tf.train.AdamOptimizer(0.1).minimize(loss_)

with tf.Session() as sess:

    # initialize variables
    tf.global_variables_initializer().run()
    
    # loop over the training data
    for i in range(num_iterations):
        idx = i % X_train.shape[0]
        _, l = sess.run([train_step, loss_], 
                        feed_dict={x_: X_train[idx].reshape(1, X.shape[1]), 
                                   y_: y_train[idx].reshape(1, 1)})
        
        # print out performance every 20 iterations
        if i % (num_iterations / 20) == 0:
            loss_train, accuracy_train = sess.run([loss_, accuracy_],
                                                  feed_dict={x_: X_train,
                                                             y_: y_train.reshape((len(y_train), 1))})
            loss_test, accuracy_test = sess.run([loss_, accuracy_],
                                                feed_dict={x_: X_test,
                                                           y_: y_test.reshape((len(y_test), 1))})            
            print "it: %04d: loss(train) = %.3f, acc(train) = %.3f, loss(test)= %.3f, acc(test) = %.3f" % \
                (i, loss_train, accuracy_train, loss_test, accuracy_test)
    
    # compute estimated probabilities on the grid for visualization
    G_prob = sess.run(p_, feed_dict={x_: G})
    G_prob = G_prob.reshape(xx.shape)

# plot decision boundary
plt.figure(figsize=(10, 10))
plt.contourf(xx, yy, G_prob, cmap=plt.cm.RdBu_r, alpha=.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, linewidths=0)
plt.axes().set_aspect('equal')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max);